In [1]:
# DPLL

Code hieronder is puur om de literal te kiezen die het meeste voorkomt 

In [ ]:
from collections import defaultdict 

def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1
            
    mostLiterals = max(counts, key=counts.get)
    amount = counts[mostLiterals]
    #print("Most occurring literal:", mostLiterals, "with amount:", amount)
    return mostLiterals


# Mijn versie van DPLL
Mocht je willen kijken hoe mijn versie van DPLL eruit ziet


In [ ]:
from typing import List, Tuple, Dict, Optional, Iterable
from collections import defaultdict, deque
import time

Assignment = Dict[int, bool]

def parse_dimacs(input_path: str) -> Tuple[Iterable[Iterable[int]], int]:
    close = False
    if isinstance(input_path, str):
        file = open(input_path, "r")
        close = True
    else:
        file = input_path


    line = file.readline()

    components = line.strip().split(" ")

    if len(components)!= 4 or components[0]!="p" or components[1]!="cnf":
      print("Wrong file format! Expected first line to be 'p cnf NUM_VARS NUM_CLAUSES")
      exit(1)

    num_vars=int(components[2])
    num_clauses=int(components[3])

    clauses=[]

    line=file.readline()
    while(line):
       numbers = [int(x) for x in line.strip().split(" ")]

       if(numbers[-1]!=0):
          print("Wrong format! Clause lines must be terminated with a 0")

       clauses.append(numbers[:-1])

       line=file.readline()


    return clauses, num_vars


def write_dimacs(target, num_vars: int, clauses) -> None:
    """Write DIMACS CNF to a file path or file-like (stdout)."""
    close = False
    if isinstance(target, str):
        f = open(target, "w")
        close = True
    else:
        f = target
    try:
        clauses = list(clauses)
        f.write(f"p cnf {num_vars} {len(clauses)}\n")
        for cl in clauses:
            f.write(" ".join(str(l) for l in cl) + " 0\n")
    finally:
        if close:
            f.close()


def delete_tautologies(clause: list[int], clauses: list[list[int]]) -> int:
    taut_count = 0
    for literal in clause:
        opposite_literal = literal * -1
        if opposite_literal in clause:
            # delete clause from clause list
            #clauses.remove(clause)
            taut_count += 1
            return taut_count
    return taut_count

def pure_literal(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], list[int]]:
    # Nog veel werk aan de winkel
    truth_list = []

    for literal in clause:
        pure = True
        neg_lit = literal * -1
        # Check if only negative or positive in other clauses
        for loop_clause in clauses:
            # Finding prove of literal not being pure
            if neg_lit in loop_clause:
                pure = False
                break
        # Take literal out of all clauses if it is pure
        if pure:
            # print(f"literal {literal} is seen as pure")
            if not literal in truth_list:
                truth_list.append(literal)
            for loop_clause in clauses[:]:
                if literal in loop_clause:
                    clauses.remove(loop_clause)

    return clauses, truth_list



def unit_clause(clause: list[int], clauses: list[list[int]]) -> Tuple[list[list[int]], int, int]:
    if len(clause) == 1 and not clause[0] == 0:
        truth = clause[0]
        #clauses.remove(clause)
        """
        for clause_loop in clauses[:]:
            opposite = truth * -1
            if truth in clause_loop:
                clauses.remove(clause_loop)
            elif opposite in clause_loop:
                clause_loop.remove(opposite)        
        """
        new_clauses = []
        opposite = -truth

        for clause in clauses:
            if truth in clause:
                continue  # deze clause is voldaan, dus overslaan
            if opposite in clause:
                clause = [lit for lit in clause if lit != opposite]
            new_clauses.append(clause)

        clauses = new_clauses

        # Take clause out of all clauses
        """for clause_loop in clauses:
            if truth in clause_loop:
                clause_loop.remove(truth)
            elif truth * -1 in clause_loop:
                clause_loop.remove(truth * -1)
        """
        unit = 1
    else:
        unit = 0
        truth = 0
    return clauses, truth, unit



def simplify(clauses: List[List[int]]) -> Tuple[List[List[int]], List[int]]:
    """
    Snelle simplificatie:
      - verwijder tautologieën
      - unit propagation (herhaald, met queue)
      - pure literal elimination (globaal per iteratie)
    Geeft (nieuwe_clauses, truth_list) terug.
    """

    # verzameling van alle geforceerde waar-literals
    truth_set = set()

    # we werken op een lokale kop zodat het origineel niet stuk gaat
    clauses = [list(cl) for cl in clauses]

    changed = True
    while changed:
        changed = False

        # -------------------------
        # 1. Tautologieën + directe unit clauses
        # -------------------------
        unit_queue = deque()
        new_clauses: List[List[int]] = []

        for clause in clauses:
            # duplicates binnen clausule verwijderen
            lits = set(clause)

            # tautologie? (x en -x in dezelfde clausule)
            if any(-lit in lits for lit in lits):
                # sla deze clausule over
                changed = True
                continue

            # unit clausule?
            if len(lits) == 1:
                lit = next(iter(lits))
                unit_queue.append(lit)
                changed = True
                continue

            new_clauses.append(list(lits))

        clauses = new_clauses

        # -------------------------
        # 2. Unit propagation (met queue)
        # -------------------------
        while unit_queue and clauses:
            u = unit_queue.popleft()

            # conflict: zowel u als -u?
            if -u in truth_set:
                # formule is inconsistent -> leeg clause toevoegen
                return ([[]], list(truth_set))

            if u in truth_set:
                continue  # al verwerkt

            truth_set.add(u)
            changed = True

            new_clauses = []
            for clause in clauses:
                # clause is voldaan?
                if u in clause:
                    continue

                # tegensgestelde literal verwijderen
                if -u in clause:
                    new_clause = [lit for lit in clause if lit != -u]
                    if len(new_clause) == 0:
                        # lege clausule -> UNSAT signaal voor DPLL
                        new_clauses.append(new_clause)
                    elif len(new_clause) == 1:
                        # nieuwe unit literal gevonden
                        unit_queue.append(new_clause[0])
                        new_clauses.append(new_clause)
                    else:
                        new_clauses.append(new_clause)
                else:
                    new_clauses.append(clause)

            clauses = new_clauses

        # Als we al een lege clausule hebben, hoeven we niet verder
        if any(len(cl) == 0 for cl in clauses):
            break

        # -------------------------
        # 3. Pure literal elimination
        # -------------------------
        # tel voorkomen van iedere literal
        lit_count = defaultdict(int)
        for clause in clauses:
            for lit in clause:
                lit_count[lit] += 1

        # pure literals = lit met count>0 en count(-lit)==0
        pure_lits = [
            lit for lit, cnt in lit_count.items()
            if cnt > 0 and lit_count.get(-lit, 0) == 0
        ]

        if pure_lits:
            changed = True
            pure_set = set(pure_lits)
            # voeg toe aan assignment
            for p in pure_set:
                truth_set.add(p)

            # alle clausules die een pure literal bevatten zijn voldaan -> weg
            new_clauses = []
            for clause in clauses:
                if any(lit in pure_set for lit in clause):
                    continue
                new_clauses.append(clause)
            clauses = new_clauses

    # einde while: geen veranderingen meer
    return clauses, list(truth_set)


##################################################################################
# DPLL
##################################################################################

DPLL_CALLS = 0


def choose_literal_dlis(clauses: List[List[int]]) -> int:
    counts: Dict[int, int] = defaultdict(int)
    for clause in clauses:
        for literal in clause:
            counts[literal] += 1
            
    mostLiterals = max(counts, key=counts.get)
    amount = counts[mostLiterals]
    #print("Most occurring literal:", mostLiterals, "with amount:", amount)
    return mostLiterals

def dpll(
    clauses: List[List[int]],
    num_vars: int,
    assignment: Optional[List[int]] = None
) -> Tuple[bool, List[int]]:
    
    # Check how many times DPLL is called
    global DPLL_CALLS
    DPLL_CALLS += 1
    print("DPLL call number:", DPLL_CALLS)
    if assignment is None:
        assignment = []

    # Simplify the clauses
    simplified_clauses, implied_truths = simplify(clauses)

    # Add implied truths to assignment
    full_assignment = assignment.copy()
    for literal in implied_truths:
        if -literal in full_assignment:
            # conflict in assignment
            return False, []
        if literal not in full_assignment:
            full_assignment.append(literal)

    # Check for empty clause set -> return UNSAT
    if any(len(cl) == 0 for cl in simplified_clauses):
        return False, []

    # Check if all clauses are satisfied
    if not simplified_clauses:
        return True, full_assignment

    # DLIS step -> choose literal to split on (assign true or false)        
    split_lit = choose_literal_dlis(simplified_clauses)

    sat, model = dpll(
    simplified_clauses + [[split_lit]],
    num_vars,
    full_assignment + [split_lit])

    if sat:
        return True, model

    sat,model = dpll(
    simplified_clauses + [[-split_lit]],
    num_vars,
    full_assignment + [-split_lit])

    return sat, model 


if __name__ == "__main__":
    # clauses, num_vars = parse_dimacs("inst16.cnf")
    # choose_literal_dlis(clauses)
    # sat, assignment = dpll(clauses, num_vars)
    # if sat:
    #     print("SAT")
    #     print(assignment)   
    # else:
    #     print("UNSAT")
    clauses, num_vars = parse_dimacs("inst16.cnf")

    import time
    start = time.perf_counter()
    sat, assignment = dpll(clauses, num_vars)
    end = time.perf_counter()

    print("SAT:", sat)
    print("DPLL calls:", DPLL_CALLS)
    print("Time:", end - start, "sec")  

1 2 3 4 5 6 7 8 9 0
